In [ ]:
import os,re, pickle, random
import numpy as np
import pandas as pd
from itertools import chain
from bs4 import BeautifulSoup
from xgboost import XGBRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import keras
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import MeanAbsoluteError

import matplotlib
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error, roc_curve, auc
matplotlib.rc('xtick', labelsize=16) 
matplotlib.rc('ytick', labelsize=16) 

In [ ]:
# Go to that directory where log files (single stuck at fault manifest as Missing/Delayed and Spurious) are saved
cd 32x32/ 

In [ ]:
def getInfo(file, ip_spks):

    """
    Args:
    file : Log files 
    ip_spks : Number of input neurons or pre-synaptic neurons

    return :
    df : Dataframe
    
    """

    with open(file) as f:
        soup = BeautifulSoup(f.read(), "html.parser")

    df = pd.DataFrame()

    # Making column names
    cols = ["Time(s)"]
    for i in range(ip_spks):
      cols.append("spk_ip_" + str(i))
      cols.append("#Spikes_" + str(i))
      cols.append("ISI_" + str(i))

    for i in range(ip_spks):
      cols += ["spk_out_" + str(i), "#Spikes_out_"+str(i), "ISI_out_"+str(i)]

    time = [int(i.split(":")[1]) for i in re.findall(r"Time : \d+", soup.text)]

    df["Time(s)"] = time

    # NeuronID info for each spike generating neuron
    n_id_info = re.findall("NeuronID: (.+)", soup.text)

    spikes_info = [list(map(int,re.findall(r'(\d+)', i))) for i in n_id_info]
    
    n_id_info_v1 = []

    for idx in range(0, len(spikes_info), ip_spks):
      var = list(chain.from_iterable((x[0],len(x)-1, x[1:]) for x in spikes_info[idx: idx + ip_spks]))
      n_id_info_v1.append(var)
  
    df[cols[1:-3 *ip_spks]] = n_id_info_v1

    # Output column info
    out = re.findall("Output Neuron at column (.+)", soup.text)

    spikes_info_out = [list(map(int, re.findall(r'(\d+)', i))) for i in out]
    out_info = []

    for idx in range(0, len(spikes_info_out), ip_spks):
      var1 = list(chain.from_iterable((x[0],len(x)-1, x[1:]) for x in spikes_info_out[idx: idx + ip_spks]))
      out_info.append(var1)

    # Checking #instances
    assert len(out_info) == len(time) == len(n_id_info_v1)

    df[cols[-3*ip_spks:]] = out_info


    return df

In [ ]:
# To compute average ISI (Inter-spiking interval)
def getAvgIsi(x):

  if len(x) == 0:
    return 0

  elif len(x) == 1:
    return x[0]%1000

  else:
    x = [i%1000 for i in x]
    if x[0] == 0:
      x[0] = 1
    return sum([(x[i+1])- (x[i]) for i in range(len(x)-1)])/(len(x)-1)

In [ ]:
# To compute COV (coefficient of variation) or Variance
def cov_or_variance(x, type_metric = "cov"):

  if len(x) == 0:
    return 0

  elif len(x) == 1:
    return x[0]%1000

  else:
    x = [i%1000 for i in x]
    if x[0] == 0:
      x[0] = 1
    imd = np.array([x[i]%1000 for i in range(len(x))])
    mu = np.mean(imd)
    
    if type_metric == "cov": 
      # coefficient of variation
      cov = np.sqrt(np.sum(np.square(imd - mu))/ (len(imd) - 1))/mu
      return cov
    else:
      # Variance of spikes
      var_spk = np.sum(np.square(imd - mu))/ (len(imd) - 1)
      return var_spk

In [ ]:
# # To make a directory to save all log files (Missing/Delayed and Spurious) with different utilizations for each pre-synaptic neuron or input neuron
utilization = range(10,51,10) 

for ut in utilization:
  os.mkdir("32x32/"+ str(ut) + "_percent/files_32_raw")

In [ ]:
def SaveRawFile(dataframe, fol, savePath, cp):
  """
  Args :
  dataframe: Dataframe
  fol : File name of post-synaptic neuron ID
  savePath : The path to save .xlsx file 
  cp :  Crosspoint
  
  """

  columns = dataframe.columns
  f = pd.DataFrame()

  idx = 0
  for e in columns:
    
    if e.startswith("ISI_"):

      if e.startswith("ISI_out_"):
          f[e] =  dataframe[e]
          f['Avg_ISI_out_'+e.split("ISI_out_")[-1]] =  dataframe[e].apply(lambda x: getAvgIsi(x))
          f['Cov_spk_out_'+e.split("ISI_out_")[-1]] =  dataframe[e].apply(lambda x: cov_or_variance(x, type_metric = "cov"))
          f['Var_spk_out_'+e.split("ISI_out_")[-1]] =  dataframe[e].apply(lambda x: cov_or_variance(x, type_metric = "var"))
        
      else:
        f[e] =  dataframe[e]
        f['Avg_ISI_'+str(idx)] =  dataframe['ISI_' + str(idx)].apply(lambda x: getAvgIsi(x))
        f['Cov_spk_ip_'+str(idx)] =  dataframe['ISI_' + str(idx)].apply(lambda x: cov_or_variance(x, type_metric = "cov"))
        f['Var_spk_ip_'+str(idx)] =  dataframe['ISI_' + str(idx)].apply(lambda x: cov_or_variance(x, type_metric = "var"))
        idx += 1

    else:
      f[e] = dataframe[e]

  f.to_excel(savePath + fol + "_" + str(cp) + ".xlsx", index = False)
  

# Saving log files in the desirable and readable format

In [ ]:
"""
cp : Crosspoints
util : Utilization (10% or 20% or 30% or 40% or 50% of the crossbar array (in terms of the crosspoints))
log_path : The path from where the log file will be read.
save_path : The path where the log file will be saved.

"""

cp_util = [(4,10), (7,20), (10,30), (13,40), (16,50)]
type_fol = ["Output_Missing", "Output_Spurious"] 

for params in cp_util:
  cp,util = params

  for fol in type_fol:
    logs_path = "32x32/" + str(util) + "_percent/" + fol + "/log_weight.txt"
    savePath = "32x32/" + str(util) + "_percent/" + "files_32_raw/"

    ## Saving raw data in excel file
    print(f"{util}% utilization ({cp} crosspoints) for {fol}.")
    df = getInfo(logs_path, 32)
    SaveRawFile(df, fol, savePath, cp)

10% utilization (4 crosspoints) for Output_Missing.
10% utilization (4 crosspoints) for Output_Spurious.
20% utilization (7 crosspoints) for Output_Missing.
20% utilization (7 crosspoints) for Output_Spurious.
30% utilization (10 crosspoints) for Output_Missing.
30% utilization (10 crosspoints) for Output_Spurious.
40% utilization (13 crosspoints) for Output_Missing.
40% utilization (13 crosspoints) for Output_Spurious.
50% utilization (16 crosspoints) for Output_Missing.
50% utilization (16 crosspoints) for Output_Spurious.


In [ ]:
# Sort the name of file of post-synaptic neuron ID

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):

    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

In [ ]:
# # To create a directory to save all features (Spike rate, Avg. ISI/COV/Variance) and their ground truth (Observed ISI) 
utilization = range(10,51,10) 
crossbarName = "32x32"

for ut in utilization:
  os.mkdir(crossbarName + "/"+ str(ut) + "_percent/StackedFiles_avg_ISI")
  # os.mkdir(crossbarName + "/"+ str(ut) + "_percent/StackedFiles_cov_spk")
  # os.mkdir(crossbarName + "/"+ str(ut) + "_percent/StackedFiles_var_spk")

In [ ]:
type_fol = ["Output_Missing", "Output_Spurious"]
for ut in utilization:
  for fol in type_fol:
    os.mkdir(crossbarName + "/" + str(ut) + "_percent/StackedFiles_avg_ISI/"+fol)
    # os.mkdir(crossbarName + "/" + str(ut) + "_percent/StackedFiles_cov_spk/"+fol)
    # os.mkdir(crossbarName + "/" + str(ut) + "_percent/StackedFiles_var_spk/"+fol)

In [ ]:
"""
Extracting features (Spike rate, Avg. ISI/COV/Variance) and their ground truth (Observed ISI) and then
save in .npy format.

"""

def StackedFile(path, fileName, nG, savePath, variance_feat = False):
    """
    Args: 
    path : The path to read excel files
    nG : Number of post-synaptic neurons
    variance_feat : Avg. ISI/COV/Variance as features included or not
    fileName : An excel file (Output_Spurious, Output_Missing)
    savePath : The path where the excel files will be saved in a numpy format in a parsed version based on cp.

    """

    """
    Predicting only average ISI_out, not #Spikes_out
    """
    

    # Number of crosspoints (numCp)
    numCp = int(fileName.split("_")[-1][:-5]) 
    print(f"File Name : {fileName} | Number of crosspoints : {numCp}")

    file = pd.read_excel(path + fileName)

    cols_drop = []

    for id in range(nG):
      cols_drop.append("spk_ip_" + str(id))
      cols_drop.append("spk_out_" + str(id))
      cols_drop.append("ISI_" + str(id))
      cols_drop.append("ISI_out_" + str(id))
      cols_drop.append("#Spikes_out_" + str(id))

      if variance_feat == "var":
        cols_drop.append("Cov_spk_ip_" + str(id))
        cols_drop.append("Cov_spk_out_" + str(id))
        cols_drop.append("Avg_ISI_" + str(id))
        cols_drop.append("Avg_ISI_out_" + str(id))

      elif not variance_feat: 
        cols_drop.append("Cov_spk_ip_" + str(id))
        cols_drop.append("Cov_spk_out_" + str(id))
        cols_drop.append("Var_spk_ip_" + str(id))
        cols_drop.append("Var_spk_out_" + str(id))
      else:
        cols_drop.append("Avg_ISI_" + str(id))
        cols_drop.append("Avg_ISI_out_" + str(id))
        cols_drop.append("Var_spk_ip_" + str(id))
        cols_drop.append("Var_spk_out_" + str(id))
        

    cols_drop.extend(['Time(s)'])

    file.drop(cols_drop, axis=1, inplace=True)

    file_v1 = file.reset_index(drop=True)

    if variance_feat == "var":
        y_cols = ["Var_spk_out_" + str(idx) for idx in range(nG)]
    elif not variance_feat: 
        y_cols = ["Avg_ISI_out_" + str(idx) for idx in range(nG)]
    else:
        y_cols = ["Cov_spk_out_" + str(idx) for idx in range(nG)]
        

    X_train  = file_v1.drop(y_cols, axis=1).values
    y_train =  file_v1[y_cols].values
    
    X_train_v1 = np.concatenate([X_train[:, :numCp*2], np.zeros((X_train.shape[0], X_train.shape[1] - (numCp*2)))], axis =1)

    
    X_tr, y_tr =  X_train_v1, y_train

      
    with open(savePath + "/X_train_stacked.npy", 'wb') as f:
      np.save(f, X_tr)

    for col in range(nG):
      with open(savePath +"/y_train_stacked_" + str(col) + ".npy", 'wb') as f:
        np.save(f, y_tr[:,col])


In [ ]:
def run(utilization, variance_feat, nG, crossbarName):

  for ut in utilization:
    type_files = [i for i in os.listdir(crossbarName + "/"+ str(ut) + "_percent/files_"+str(nG)+"_raw/") if i.endswith("xlsx")]
    for file in type_files:
      fol = "_".join(file.split("_")[:-1])
      if variance_feat == "var":
        savePath = crossbarName + "/"+ str(ut) + "_percent/StackedFiles_var_spk/"+fol
      elif not variance_feat:
        savePath = crossbarName + "/"+ str(ut) + "_percent/StackedFiles_avg_ISI/"+fol
      else:
        savePath = crossbarName + "/"+ str(ut) + "_percent/StackedFiles_var_spk/"+fol

      StackedFile(crossbarName + "/"+ str(ut) + "_percent/files_"+str(nG)+"_raw/", file, nG, savePath, variance_feat = variance_feat)

In [5]:
# Considering #Spikes & Avg ISI as features

variance_feat = False 
crossbarName = "32x32"
utilization = range(10,51,10)
nG = 32

run(utilization, variance_feat, nG, crossbarName)

File Name : Output_Missing_4.xlsx | Number of crosspoints : 4
File Name : Output_Spurious_4.xlsx | Number of crosspoints : 4
File Name : Output_Missing_7.xlsx | Number of crosspoints : 7
File Name : Output_Spurious_7.xlsx | Number of crosspoints : 7
File Name : Output_Missing_10.xlsx | Number of crosspoints : 10
File Name : Output_Spurious_10.xlsx | Number of crosspoints : 10
File Name : Output_Missing_13.xlsx | Number of crosspoints : 13
File Name : Output_Spurious_13.xlsx | Number of crosspoints : 13
File Name : Output_Missing_16.xlsx | Number of crosspoints : 16
File Name : Output_Spurious_16.xlsx | Number of crosspoints : 16


In [3]:
# Considering #Spikes & Variance as features

# variance_feat = "var" 
# crossbarName = "32x32"
# utilization = range(10,51,10)
# nG = 32

# run(utilization, variance_feat, nG, crossbarName)

File Name : Output_Missing_4.xlsx | Number of crosspoints : 4
File Name : Output_Spurious_4.xlsx | Number of crosspoints : 4
File Name : Output_Missing_7.xlsx | Number of crosspoints : 7
File Name : Output_Spurious_7.xlsx | Number of crosspoints : 7
File Name : Output_Missing_10.xlsx | Number of crosspoints : 10
File Name : Output_Spurious_10.xlsx | Number of crosspoints : 10
File Name : Output_Missing_13.xlsx | Number of crosspoints : 13
File Name : Output_Spurious_13.xlsx | Number of crosspoints : 13
File Name : Output_Missing_16.xlsx | Number of crosspoints : 16
File Name : Output_Spurious_16.xlsx | Number of crosspoints : 16


In [4]:
# Considering #Spikes & COV as features

# variance_feat = True 
# crossbarName = "32x32"
# utilization = range(10,51,10)
# nG = 32

# run(utilization, variance_feat, nG, crossbarName)

File Name : Output_Missing_4.xlsx | Number of crosspoints : 4
File Name : Output_Spurious_4.xlsx | Number of crosspoints : 4
File Name : Output_Missing_7.xlsx | Number of crosspoints : 7
File Name : Output_Spurious_7.xlsx | Number of crosspoints : 7
File Name : Output_Missing_10.xlsx | Number of crosspoints : 10
File Name : Output_Spurious_10.xlsx | Number of crosspoints : 10
File Name : Output_Missing_13.xlsx | Number of crosspoints : 13
File Name : Output_Spurious_13.xlsx | Number of crosspoints : 13
File Name : Output_Missing_16.xlsx | Number of crosspoints : 16
File Name : Output_Spurious_16.xlsx | Number of crosspoints : 16


In [ ]:
def getBinary(y_true, y_pred, margin):
  
  """
  It will binarize the continuous values which are y_true and y_pred with
  the help of margin and threshold to plot ROC (Receiver operating characteristic).

  Args:
  y_true : Observed avg. ISI (continuous values)
  y_pred : Predicted avg. ISI (continuous values)
  margin : The margin is the obtained optimal MAE or a user can take loose margin as per his choice to incur more false alaram or false fault

  return:
  y_t : Binarized y_true
  y_p : Binarized y_pred

  The reason to opt for ROC curve because it offers an elegant way to
  plot true fault detection rate versus false fault detection rate.
 
  """

  thresh = pd.Series(y_true).median()   # We have taken median as a threshold because it is not impacted by the outliers
  y_t = []
  y_p = []

  for t,p in zip(y_true, y_pred):
    if np.abs(t - p) <= margin:
      if t> thresh:
        y_t.append(1)
        y_p.append(1)
      else:
        y_t.append(0)
        y_p.append(0)

    else:
      if (t > thresh) and (p > thresh):  # ex: t=60, p=80 but thresh = 55 --> t,p -> 1, therefore one of them has to be opposite to another because abs(t-p)>15.
        y_t.append(1)
        y_p.append(0)

      elif (t > thresh) and (p <= thresh):  # ex: t=60, p=40 but thresh = 55 --> t->1,  p-> 0, which satisfies the condition, one has to be opposite of another.
        y_t.append(1)
        y_p.append(0)

      elif (t <= thresh) and (p > thresh):  # ex: t=30, p=57 but thresh = 55 --> t->0,  p-> 1,
        y_t.append(0)
        y_p.append(1)

      elif (t <= thresh) and (p <= thresh): # ex: t=25, p=45 but thresh = 55 --> t->0, p-> 1, therefore one of them has to be opposite to another.
        y_t.append(0)
        y_p.append(1)
      

  y_t = np.array(y_t)
  y_p = np.array(y_p)

  return y_t, y_p

In [ ]:
def plotROC(y_true, y_pred, reg, margin, color):

  """
  To plot ROC

  Args: 
  y_true : Observed avg. ISI
  y_pred : Predicted avg. ISI
  reg :  Regressor Name
  margin : The margin is the obtained optimal MAE or a user can take loose margin as per his choice to incur more false alaram or false fault
  color : Color

  return:
  y_t : Binarized y_true
  y_p : Binarized y_pred

  """

  y_t, y_p = getBinary(y_true, y_pred, margin)
  
  fpr, tpr, _ = roc_curve(y_t, y_p )
  roc_auc = auc(fpr, tpr)

  print()
  print("False positive rate : ", fpr[1])
  print("True positive rate : ", tpr[1])
  print("ROC Area under curve : ", roc_auc)

  plt.figure(figsize = (6,6))
  lw = 2
  plt.plot(fpr, tpr, color= color, lw=lw, label=f"ROC curve (area = %0.2f) for {reg}." % roc_auc,linewidth=4)
  plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel("False Positive Rate", fontsize=15)
  plt.ylabel("True Positive Rate", fontsize=15)
  plt.title("Receiver operating characteristic example")
  plt.legend(loc="lower right")
  plt.show()

  return y_t, y_p

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def getPrediction(nG, ip_shape, scaling_mode, file_Type, crossbarName, model_type, model_name, optimal_maes, models_list, variance_feat =False):

  """
  Args:
  nG : Number of post-synaptic neurons
  scaling_mode : Type of scaling technique
  file_type : "Missing/Delayed" file or "Spurious" file
  crossbarName : Crossbar type 32x32 or 64x64 or 128x128
  model_type : "Tree" based or "NN" based
  model_name : which model to run
  models_list : List of model names
  optimal_maes : Obtained optimal MAEs for all columns
  ip_shape : Dimension of an example (Mainly for CNN)
  variance_feat : Coefficient of variation/Variance/Avg ISI as features included or not

  print : 
  Fault detection rate

  """

  row, col, chn = ip_shape
  utilization = range(10,51,10)

  print("*"*25 + " "+ file_Type.split("_")[-1] +" Spikes " + "*"*25)

  for util in utilization:
    print("*"*25 + " Utilization : " + str(util) + "% " + "*"*25)
    print()
    print("Scaling Mode : ", scaling_mode)
    print("Model Name : ", model_name, "\n")

    if variance_feat == "var":
      with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_var_spk/" + file_Type + "/X_train_stacked.npy", 'rb') as f:
        X = np.load(f)
    elif not variance_feat:
      with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_avg_ISI/" + file_Type + "/X_train_stacked.npy", 'rb') as f:
        X = np.load(f)
    else:
      with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_cov_spk/" + file_Type + "/X_train_stacked.npy", 'rb') as f:
        X = np.load(f)

    prediction_wrt_column = []
    
    for id in range(nG):
      # print("#"*25 + " " + "Column" + str(id) +" " + "#"*25)

      if variance_feat == "var":
        with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_var_spk/" + file_Type +"/y_train_stacked_" + str(id) + ".npy", 'rb') as f:
          y = np.load(f)
          y = y/10000  # (for better fit)
      elif not variance_feat:
        with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_avg_ISI/" + file_Type +"/y_train_stacked_" + str(id) + ".npy", 'rb') as f:
          y = np.load(f)
          y = y/10  # (To convert into decisecond for better fit)
      else:
        with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_cov_spk/" + file_Type +"/y_train_stacked_" + str(id) + ".npy", 'rb') as f:
          y = np.load(f)

      
      X_test, y_test = X[:5000], y[:5000] # Considering 5k samples

      # Standard Scaling
      if scaling_mode == 'std':
        # print("Standard Scaling",'\n')
        if variance_feat == "var":
          with open(os.getcwd() + "/Scaler_var_spk/" + "std_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)
        elif not variance_feat:
          with open(os.getcwd() + "/Scaler_avg_ISI/" + "std_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)
        else:
          with open(os.getcwd() + "/Scaler_cov_spk/" + "std_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)
        
        x_test_scaled = scaler.transform(X_test)


      # Min-max Scaling
      elif scaling_mode == "minmax":
        # print("Minmax Scaling \n")
        if variance_feat == "var":
          with open(os.getcwd() + "/Scaler_var_spk/" + "minmax_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)
        elif not variance_feat:
          with open(os.getcwd() + "/Scaler_avg_ISI/" + "minmax_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)
        else:
          with open(os.getcwd() + "/Scaler_cov_spk/" + "minmax_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)

        x_test_scaled = scaler.transform(X_test)


      if variance_feat == "var":
        if model_type == "Tree":
          with open(os.getcwd() + "/Models_var_spk/" + model_name + "_column_" + str(id) + ".pkl", 'rb') as file:  
            reg = pickle.load(file)
        else:
          if model_name == "ANN":
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_var_spk/" + model_name + "_column_" + str(id) +".h5", compile=False)
          else:
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_var_spk/" + model_name + "_column_" + str(id) +".h5", compile=False)

      elif not variance_feat:
        if model_type == "Tree":
          with open(os.getcwd() + "/Models_avg_ISI/" + model_name + "_column_" + str(id) + ".pkl", 'rb') as file:  
            reg = pickle.load(file)
        else:
          if model_name == "ANN":
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_avg_ISI/" + model_name + "_column_" + str(id) +".h5", compile=False)
          else:
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_avg_ISI/" + model_name + "_column_" + str(id) +".h5", compile=False)        

      else:
        if model_type == "Tree":
          with open(os.getcwd() + "/Models_cov_spk/" + model_name + "_column_" + str(id) + ".pkl", 'rb') as file:  
            reg = pickle.load(file)
        else:
          if model_name == "ANN":
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_cov_spk/" + model_name + "_column_" + str(id) +".h5", compile=False)
          else:
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_cov_spk/" + model_name + "_column_" + str(id) +".h5", compile=False)


    
      if model_name == "CNN":
        test_pred = reg.predict(x_test_scaled.reshape((x_test_scaled.shape[0],row, col, chn)), verbose=0).flatten()
      elif model_name == "ANN":
        test_pred = reg.predict(x_test_scaled, verbose=0).flatten()
      else:
        test_pred = reg.predict(x_test_scaled)

      margin = optimal_maes[models_list.index(model_name)][id][-1]/2.5 # To tighten the margin
        
      y_t, y_p = getBinary(y_test, test_pred, margin=margin)

      detectecd = (sum(y_t!= y_p))/(len(y_t))

      prediction_wrt_column.append(detectecd)

    print(prediction_wrt_column)
    print(np.mean(prediction_wrt_column))
    print()

In [ ]:
# Trained models list
models_list = ["XGBRegressor", "LGBMRegressor", "CatBoostRegressor", "ANN", "CNN"]

# Loading MAEs_ISI to numpy array
with open('MAEs_ISI.npy', 'rb') as f:
    MAEs_ISI = np.load(f)

MAEs_ISI.shape

(5, 32, 3)

In [ ]:
nG = 32
ip_shape = (8,8,1)
scaling_mode = "minmax"
fol_Type = "Output_Missing"
crossbarName = "32x32"
model_type = "NN"
model_name = models_list[4]


getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_ISI, models_list, variance_feat =False)

************************* Missing Spikes *************************
************************* Utilization : 10% *************************

Scaling Mode :  minmax
Model Name :  CNN 

[0.9134, 0.9764, 0.9686, 0.9074, 0.9104, 0.9114, 0.967, 0.9558, 0.9058, 0.8848, 0.902, 0.9118, 0.8602, 0.9624, 0.967, 0.9114, 0.951, 0.9078, 0.8714, 0.9528, 0.8988, 0.8782, 0.8854, 0.8622, 0.9706, 0.8836, 0.8948, 0.8714, 0.9488, 0.9834, 0.8694, 0.9398]
0.9182875

************************* Utilization : 20% *************************

Scaling Mode :  minmax
Model Name :  CNN 

[0.8324, 0.9732, 0.9748, 0.7712, 0.7942, 0.8462, 0.9002, 0.8914, 0.7696, 0.7244, 0.774, 0.8058, 0.719, 0.9044, 0.9338, 0.8168, 0.8878, 0.7976, 0.714, 0.8856, 0.7498, 0.703, 0.7336, 0.699, 0.9434, 0.7556, 0.776, 0.7142, 0.852, 0.9586, 0.706, 0.8302]
0.81680625

************************* Utilization : 30% *************************

Scaling Mode :  minmax
Model Name :  CNN 

[0.8954, 0.9862, 0.9748, 0.8568, 0.8692, 0.8982, 0.9158, 0.8518, 0

In [ ]:
nG = 32
ip_shape = (8,8,1)
scaling_mode = "minmax"
fol_Type = "Output_Spurious"
crossbarName = "32x32"
model_type = "NN"
model_name = models_list[4]


getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_ISI, models_list, variance_feat =False)

************************* Spurious Spikes *************************
************************* Utilization : 10% *************************

Scaling Mode :  minmax
Model Name :  CNN 

[0.9656, 0.8668, 0.8718, 0.9682, 0.9662, 0.962, 0.9118, 0.9232, 0.9696, 0.976, 0.9712, 0.9694, 0.9766, 0.9132, 0.8886, 0.9712, 0.9268, 0.969, 0.9766, 0.9216, 0.9704, 0.9708, 0.973, 0.9772, 0.8748, 0.9652, 0.965, 0.9702, 0.9316, 0.8038, 0.9704, 0.9198]
0.942425

************************* Utilization : 20% *************************

Scaling Mode :  minmax
Model Name :  CNN 

[0.9164, 0.7316, 0.7106, 0.943, 0.9314, 0.8982, 0.828, 0.8428, 0.9356, 0.9684, 0.9336, 0.9062, 0.978, 0.8136, 0.7398, 0.9076, 0.8278, 0.9094, 0.9742, 0.817, 0.9406, 0.9662, 0.9628, 0.9714, 0.7074, 0.9148, 0.906, 0.9486, 0.85, 0.649, 0.9598, 0.843]
0.8791500000000001

************************* Utilization : 30% *************************

Scaling Mode :  minmax
Model Name :  CNN 

[0.8986, 0.731, 0.729, 0.9154, 0.9112, 0.8848, 0.869, 0.9108

In [ ]:
# # Loading MAEs_COV to numpy array
# with open('MAEs_COV.npy', 'rb') as f:
#     MAEs_COV = np.load(f)
# MAEs_COV.shape

In [ ]:
# models_list = ["XGBRegressor", "LGBMRegressor", "CatBoostRegressor", "ANN", "CNN"]
# nG = 32
# ip_shape = (8,8,1)
# scaling_mode = "minmax"
# fol_Type = "Output_Missing"
# crossbarName = "32x32"
# model_type = "NN"
# model_name = models_list[4]


# getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_COV, models_list, variance_feat =True)

In [ ]:
# nG = 32
# ip_shape = (8,8,1)
# scaling_mode = "minmax"
# fol_Type = "Output_Spurious"
# crossbarName = "32x32"
# model_type = "NN"
# model_name = models_list[4]


# getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_COV, models_list, variance_feat =True)

In [ ]:
# # Loading MAEs_var to numpy array
# with open('MAEs_VAR.npy', 'rb') as f:
#     MAEs_VAR = np.load(f)

# MAEs_VAR.shape

In [ ]:
# # Trained models list
# models_list = ["XGBRegressor", "LGBMRegressor", "CatBoostRegressor", "ANN", "CNN"]
# nG = 32
# ip_shape = (8,8,1)
# scaling_mode = "std"
# fol_Type = "Output_Spurious"
# crossbarName = "32x32"
# model_type = "NN"
# model_name = models_list[3]


# getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_VAR, models_list, variance_feat = "var")

In [ ]:
# models_list = ["XGBRegressor", "LGBMRegressor", "CatBoostRegressor", "ANN", "CNN"]
# nG = 32
# ip_shape = (8,8,1)
# scaling_mode = "std"
# fol_Type = "Output_Missing"
# crossbarName = "32x32"
# model_type = "NN"
# model_name = models_list[3]


# getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_VAR, models_list, variance_feat = "var")